<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/marco-canas/ml_intro/blob/main/2_planificacion/redes_neuronales_geron/chapter_10/pagina_479_edition_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
  <td>
    <a target="_blank" href="https://kaggle.com/kernels/welcome?src=https://github.com/marco-canas/ml_intro/blob/main/2_planificacion/redes_neuronales_geron/chapter_10/pagina_479_edition_3.ipynb"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" /></a>
  </td>
</table>

Aquí tienes la traducción al español del texto del capítulo 10 de *Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow* de Aurélien Géron:

---



# **MLPs para Regresión**  



En primer lugar, las redes neuronales multicapa (*MLPs*) pueden utilizarse para tareas de regresión. Si deseas predecir un único valor (por ejemplo, el precio de una casa dadas varias de sus características), solo necesitas una neurona de salida: su resultado será el valor predicho. Para regresión multivariante (es decir, predecir múltiples valores a la vez), necesitas una neurona de salida por cada dimensión de salida. Por ejemplo, para localizar el centro de un objeto en una imagen, necesitas predecir coordenadas en 2D, por lo que requieres dos neuronas de salida. Si además quieres dibujar un *bounding box* (rectángulo delimitador) alrededor del objeto, necesitas dos números más: el ancho y el alto del objeto. Así, terminarías con cuatro neuronas de salida.  

Scikit-Learn incluye la clase **`MLPRegressor`**, así que vamos a usarla para construir una MLP con tres capas ocultas de 50 neuronas cada una y entrenarla en el conjunto de datos de viviendas de California. Para simplificar, utilizaremos la función **`fetch_california_housing()`** de Scikit-Learn para cargar los datos. Este conjunto es más simple que el que usamos en el Capítulo 2, ya que solo contiene características numéricas (no incluye la característica *ocean_proximity*) y no tiene valores faltantes.  

El siguiente código comienza cargando y dividiendo el conjunto de datos, luego crea un *pipeline* para estandarizar las características de entrada antes de pasarlas al **`MLPRegressor`**. Esto es muy importante para las redes neuronales, ya que se entrenan mediante *gradiente descendente*, y como vimos en el Capítulo 4, el gradiente descendente no converge bien cuando las características tienen escalas muy diferentes.  

Finalmente, el código entrena el modelo y evalúa su error de validación. El modelo usa la función de activación **ReLU** en las capas ocultas y una variante del gradiente descendente llamada **Adam** (ver Capítulo 11) para minimizar el *error cuadrático medio (MSE)*, con un poco de regularización **ℓ₂** (controlable mediante el hiperparámetro **`alpha`**):  

```python
from sklearn.datasets import fetch_california_housing
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

mlp_reg = MLPRegressor(hidden_layer_sizes=[50, 50, 50], random_state=42)
pipeline = make_pipeline(StandardScaler(), mlp_reg)
pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_valid)
rmse = mean_squared_error(y_valid, y_pred, squared=False)  # ~0.505
```  

Obtenemos un **RMSE de validación** de aproximadamente **0.505**, comparable al rendimiento de un *Random Forest*. ¡Nada mal para un primer intento!  

Cabe destacar que esta MLP **no usa ninguna función de activación en la capa de salida**, por lo que puede generar cualquier valor. Esto suele ser aceptable, pero si necesitas garantizar que la salida siempre sea positiva, deberías usar **ReLU** o **softplus** (una variante suave de ReLU: *softplus(z) = log(1 + exp(z))*). La función softplus es cercana a 0 cuando *z* es negativo y cercana a *z* cuando es positivo.  

Por último, si quieres asegurar que las predicciones estén dentro de un rango específico, puedes usar **sigmoid** (para valores entre 0 y 1) o **tanh** (entre –1 y 1) y escalar los objetivos acordemente. Lamentablemente, **`MLPRegressor` no soporta funciones de activación en la capa de salida**.  

⚠ **Advertencia**  
Construir y entrenar una MLP estándar con Scikit-Learn en pocas líneas de código es muy conveniente, pero sus capacidades son limitadas. Por eso, más adelante en este capítulo pasaremos a **Keras**.  

La clase **`MLPRegressor`** usa el *error cuadrático medio (MSE)*, que suele ser adecuado para regresión. Sin embargo, si hay muchos *outliers* en los datos, podrías preferir el *error absoluto medio (MAE)* o la **pérdida de Huber**, que combina MSE y MAE: es cuadrática para errores menores que un umbral **δ** (típicamente 1) y lineal para errores mayores. La parte lineal la hace menos sensible a *outliers* que el MSE, mientras que la parte cuadrática permite una convergencia más rápida que el MAE. No obstante, **`MLPRegressor` solo soporta MSE**.  

La **Tabla 10-1** resume la arquitectura típica de una MLP para regresión.  

| **Hiperparámetro**         | **Valor típico**                                                                 |
|----------------------------|----------------------------------------------------------------------------------|
| N° de capas ocultas        | Depende del problema (usualmente 1 a 5)                                          |
| N° de neuronas por capa    | Depende del problema (usualmente 10 a 100)                                       |
| N° de neuronas de salida   | 1 por dimensión de predicción                                                    |
| Activación en capas ocultas| ReLU                                                                             |
| Activación en salida       | Ninguna, ReLU/softplus (salidas positivas) o sigmoid/tanh (salidas acotadas)     |
| Función de pérdida         | MSE, o Huber si hay *outliers*                                                   |

--- 

Espero que esta traducción te sea útil. Si necesitas ajustes o más detalles, ¡avísame!